## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_question_path_list = glob.glob('/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/' + '*question.h5')
model_answer_path_list = glob.glob('/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answeronly/' + '*answer.h5')
model_question_path_list.sort()
model_answer_path_list.sort()
print('Models to predict:', model_question_path_list + model_answer_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/model_fold_1_question.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/model_fold_2_question.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/model_fold_3_question.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answeronly/model_fold_1_answer.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answeronly/model_fold_2_answer.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answeronly/model_fold_3_answer.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = 'answer'

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 510

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test_question = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                                        MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)
X_test_answer = compute_input_arays_1(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                      MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
def question_model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS_QUESTION, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

def answer_model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS_ANSWER, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for index, model_path in enumerate(model_question_path_list):
    model_question = question_model_fn()
    model_answer = answer_model_fn()
    model_question.load_weights(model_path)
    model_answer.load_weights(model_answer_path_list[index])
    
    question_preds = model_question.predict(X_test_question)
    answer_preds = model_answer.predict(X_test_answer)
    Y_test += np.hstack([question_preds, answer_preds]) / len(model_question_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.935369,0.613000,0.157765,0.565073,0.625069,0.521756,0.662141,0.635480,0.645434,...,0.883141,0.926070,0.591048,0.955719,0.961784,0.849487,0.033994,0.063539,0.832078,0.889269
1,46,0.840001,0.510596,0.011727,0.761377,0.716234,0.896264,0.550075,0.419652,0.134163,...,0.602531,0.940935,0.672303,0.974305,0.981091,0.883509,0.933075,0.105300,0.061981,0.886651
2,70,0.870393,0.582569,0.033468,0.737148,0.821371,0.860227,0.589866,0.506232,0.202224,...,0.812536,0.938154,0.610027,0.974604,0.971250,0.857755,0.070243,0.079184,0.917807,0.882611
3,132,0.850551,0.415217,0.007202,0.719844,0.749118,0.903134,0.525262,0.401060,0.162609,...,0.658481,0.952358,0.698954,0.977031,0.985931,0.908319,0.808967,0.162779,0.604611,0.893952
4,200,0.887118,0.414602,0.050023,0.777806,0.762347,0.849908,0.653987,0.561541,0.142866,...,0.624554,0.883307,0.647215,0.946776,0.948658,0.811269,0.229443,0.136856,0.467765,0.900068


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.874672,0.542978,0.037522,0.717718,0.775777,0.820432,0.572588,0.470350,0.264392,...,0.761111,0.932470,0.660940,0.966181,0.973309,0.875430,0.529510,0.130247,0.491861,0.903843
std,2812.670060,0.040972,0.110307,0.048577,0.080031,0.082910,0.117182,0.051574,0.086376,0.187798,...,0.081894,0.030519,0.045850,0.013841,0.015180,0.044237,0.325972,0.052866,0.283292,0.028186
min,39.000000,0.751233,0.348220,0.004598,0.316731,0.493862,0.360521,0.481009,0.336215,0.024126,...,0.577080,0.743172,0.472188,0.866134,0.898190,0.611544,0.005319,0.012336,0.027655,0.749860
25%,2572.000000,0.844301,0.451128,0.010530,0.673397,0.733564,0.782945,0.530048,0.404944,0.114498,...,0.695342,0.920078,0.631582,0.960427,0.967812,0.852754,0.230091,0.091380,0.237232,0.888916
50%,5093.000000,0.874199,0.532720,0.016775,0.725954,0.773395,0.863577,0.565573,0.441043,0.200618,...,0.761933,0.939261,0.660816,0.969092,0.977850,0.884863,0.582392,0.127851,0.474052,0.907733
75%,7482.000000,0.905224,0.627846,0.041727,0.764682,0.831170,0.899042,0.609763,0.512959,0.387128,...,0.832497,0.954096,0.686137,0.975955,0.983668,0.906159,0.839550,0.163300,0.763110,0.922755
max,9640.000000,0.960932,0.839610,0.332040,0.921182,0.956516,0.953418,0.714501,0.762564,0.833807,...,0.915278,0.974841,0.794932,0.989181,0.991120,0.948636,0.966241,0.313752,0.976121,0.950988
